### Read sample data

The test last for 30 minutes, there are a total of 2 - 4 channels to be measured. Measurement is done every 20 seconds on all channels. Currently we have 2 channel, but in the future we will have 4 channel.

At each measuremnt, a total of 120 data point is collected. Measurement on 1 channel normally takes less than 1-2 seconds.

This sample data is the data collected on 1 channel during the test. It is a list of 90 measurements. For each measurement, it consits of a start/end potential for this measurement and 120 current data.

For example, the first measurement: 

```
[
    [
      -0.407,
      0.193
    ],
    [
      7.952,
      9.462,
      9.058,
      .
      .
      .
    ],
    ...
```

The starting potential is -.407V, ending potential is 0.193V, 120 data point is collected. Current values are 7.952... etc.

In [ ]:
# TWXY serial port data txt file
data2 = r"C:\Users\hui\Work\HuiWork\Covid_Reader\TWXY\dat_2.txt"
data4 = r"C:\Users\hui\Work\HuiWork\Covid_Reader\TWXY\dat_4.txt"



In [ ]:
# convert txt file to [[[vS,vE],[A,A,A]],...]
with open(data4,'rt') as f:
    txt = f.read()

import json
jsons = txt.replace('\n','').split('*')
alldata=[]
for i in jsons:
    if (i.strip()):
        j = json.loads(i)
        current = j.get('c',[])
        if len(current) > 50:
            alldata.append([[-0.6,0.1],current])
channel = 2
data = alldata[channel::4]


In [ ]:
len(alldata)

In [ ]:
# load the data.
# you can change './sample.json' to your own JSON data file.
# the sample json data is using a non-standard affix because of my gitignore.
import json
# data = json.load(open('./sample.json.txt'))
data = json.load(open('./exportData/positive1.json.txt'))

### Data format

In [ ]:
# the totoal time points of the data:
print(f'There are a total of {len(data)} measurements.')

print(f'The start and ending potential of the first measurement is {data[0][0][0]}V and {data[0][0][1]}V.')

print(f'The first measurement result: {data[0][1]}')
print(f"The first measurement result's length is {len(data[0][1])}")

In [ ]:
### show one of the data and the format to be returned from M355
index = -1
print('One of the returned repsonse from M355 is:')
print(json.dumps({"r":data[index][1]},separators=(',',':')) + '*')

In [ ]:
### plot the raw data
from util import plotFit
import numpy as np
index = -1
potentials = np.linspace(*data[index][0],len(data[index][1]))
currents = data[index][1]
print('This is how the raw data looks:')
plotFit(potentials,currents)

### perform data analysis to find the measurement value at each measurement

The operation is done in real time, whenever a measurement is finished, the raw data is processed to generate the fitting result. The raw data can be discarded, only store the fitting result.



In [ ]:
# the peak fitting algorithm is in the util module. 
# you can look in to details. 
# certain algorithms from python packages are used. We need to migrate those algorithms as well.

from util import myfitpeak,plotFit
from util import *
import numpy as np

fits = []
for v,a in data:
    fits.append(myfitpeak(np.linspace(*v,len(a)),a))


In [ ]:
print(json.dumps(fits[-1],indent=2))

### Fitting result
- `pc` is the peak high in the figure below.(green verticle line). This is the value we use for downstream calling.
- `fx, fy` are the cordinates of the peak base. (left and right intersection point of the orange line)
- `pv` is the peak center.
- `err` is a estimation of how close the peak is to a normal distribution. 

In [ ]:
# This is demonstrating how the raw measurement and one of fitting result looks like 
print('This is demonstrating how the raw measurement and one of fitting result looks like:')
fig,axes = plt.subplots(4,4,figsize=(24,16))
axes = [i for j in axes for i in j]
for a,i in enumerate(range(0,80,5)):
    index = i
    potentials = np.linspace(*data[index][0],len(data[index][1]))
    currents = data[index][1]
    plotFit(potentials , currents, fits[index] ,ax= axes[a], title=f"{index}th fit pc={fits[index]['pc']:.3f}")


### Perform the result calling from the time course result

The `pc` value at each measurement from the previous step is used to predict whether the channel is positive or negative.

In [ ]:
# Here is how the time course curve normally looks like
# plotting is just for show here.
import matplotlib.pyplot as plt
plt.plot( np.linspace(0,30,len(fits)) ,  [i['pc'] for i in fits], '.')
plt.ylabel('PC / uA')
plt.ylim(0,25)
plt.xlabel('Time / minutes')
plt.title('Time course of PC of  channel 3')
plt.show()

In [ ]:
from util import hCtTPredictT,convert_list_to_X

# t is the time points, the measurement is taken over 30 minutes, and a total of len(fits) measurements.
t = np.linspace(0,30,len(fits))
# c is all the `pc` in fitting result
c = [i['pc'] for i in fits]
data = [[t,c]]

# the convert_list_to_X is just to transform the data to the shape that works with the scipy pipeline.
X = convert_list_to_X(data)

# hCtPredictT is the pipeline that transforms and fits the data to give calling result.
result = hCtTPredictT.transform(X)

call, Ct, prominance, signal_drop = result[0]

print(f"The result is {'Positive' if call else 'Negative'}.")
print(f"The reaction Ct is {Ct:.2f} minutes.")
print(f"The prominance is {prominance:.4f}.")
print(f"The signal drop is {signal_drop:.4f}.")

### You can look into the `hCtTPredictT` pipeline to see what calculations are done.

The calculations utilizes some python packages, these functions need to be migrated.

In [ ]:

smoothT = Pipeline([
    ('smooth', Smoother(stddev=2, windowlength=11, window='hanning')),
    ('normalize', Normalize(mode='mean', normalizeRange=(normStart, normEnd))),
    ('truncate', Truncate(cutoffStart=cutoffStart, cutoffEnd=cutoffEnd, n=90)),
    ('remove time', RemoveTime()),
])
smoothed_X = smoothT.transform(X)

deriT = Pipeline([
    ('smooth', Smoother(stddev=2, windowlength=11, window='hanning')),
    ('normalize', Normalize(mode='mean', normalizeRange=(normStart, normEnd))),
    ('truncate', Truncate(cutoffStart=cutoffStart, cutoffEnd=cutoffEnd, n=90)),
    ('Derivitive', Derivitive(window=31, deg=3)),
    # ('remove time',RemoveTime()),
])
deri_X = deriT.transform(X)



hCtT = Pipeline([
    ('smooth', Smoother(stddev=2, windowlength=11, window='hanning')),
    ('normalize', Normalize(mode='mean', normalizeRange=(normStart, normEnd))),
    ('truncate', Truncate(cutoffStart=cutoffStart, cutoffEnd=cutoffEnd, n=90)),
    ('Derivitive', Derivitive(window=31, deg=3,deriv=1)),
    ('peak', FindPeak()),
    ('logCt',HyperCt(offset=0.05)),
    
])
hCtT_X = hCtT.transform(X)

hCtTPredictT = Pipeline([
    ('smooth', Smoother(stddev=2, windowlength=11, window='hanning')),
    ('normalize', Normalize(mode='mean', normalizeRange=(normStart, normEnd))),
    ('truncate', Truncate(cutoffStart=cutoffStart, cutoffEnd=cutoffEnd, n=90)),
    ('Derivitive', Derivitive(window=31, deg=3)),
    ('peak', FindPeak()),
    ('logCt',HyperCt(offset=0)),
    ('predictor',CtPredictor(ct=22,prominence=0.22,sd=0.05))
])
hCtpred_X = hCtTPredictT.transform(X)


In [ ]:

col = 1
row = 1
print(f'Generating curve plots in a {row} x {col} Grid')
fig, axes = plt.subplots(row, col, figsize=(col*4, row*3))

i=0
ax = axes
# ax.set_ylim([0,1.3])

smoothed_c = smoothed_X[i]
t,deri,_ =  deri_X[i]
left_ips,peak_prominence,peak_width, *sd= hCtT_X[i]    

curvePeakRange = findTimeVal(t,smoothed_c,left_ips,peak_width)
xvals = np.linspace(t[0],t[-1],len(deri))


# hyper ct
hyperline = HyperCt.hyperF(None,hCtT_X[i][-4:-1])
hyperCt = hCtT_X[i][-1]

# plot smoothed current
ax.plot(xvals,smoothed_c,color='red')
# plot the signal drop part
ax.plot(np.linspace(left_ips,left_ips+peak_width,len(curvePeakRange)) ,curvePeakRange,linewidth=4,alpha=0.75 )
# plot plot the derivative peaks
ax.plot(xvals,(deri - np.min(deri) ) / (np.max(deri) -np.min(deri) ) * (np.max(smoothed_c)-np.min(smoothed_c)) + np.min(smoothed_c),'--',alpha=0.8)
# ax.plot(xvals,fitres(xvals),'b-.')
# ax.plot(xvals,thresholdline(xvals),'b-.',alpha=0.7)
# ax.plot([thresholdCt,thresholdCt],[0,2],'k-')

# plot hyper fitting line
ax.plot(xvals,hyperline(xvals),'k--',alpha=0.7)
ax.plot([hyperCt,hyperCt],[0.5,1.1],'k--',alpha=0.7)
    
plt.tight_layout()


In [ ]:
#plot the deri peak only

col = 1
row = 1
print(f'Generating curve plots in a {row} x {col} Grid')
fig, axes = plt.subplots(row, col, figsize=(col*4, row*3))

i=0
ax = axes
# ax.set_ylim([0,1.3])

smoothed_c = smoothed_X[i]
t,deri,_ =  deri_X[i]
left_ips,peak_prominence,peak_width, *sd= hCtT_X[i]    

curvePeakRange = findTimeVal(t,smoothed_c,left_ips,peak_width)
xvals = np.linspace(t[0],t[-1],len(deri))


# hyper ct
hyperline = HyperCt.hyperF(None,hCtT_X[i][-4:-1])
hyperCt = hCtT_X[i][-1]

# plot plot the derivative peaks
ax.plot(xvals,deri*100,'--',alpha=0.8)
# ax.plot(xvals,fitres(xvals),'b-.')
# ax.plot(xvals,thresholdline(xvals),'b-.',alpha=0.7)
# ax.plot([thresholdCt,thresholdCt],[0,2],'k-')
plt.tight_layout()

In [ ]:

deri*100



In [ ]:
(deri.max() - deri[0]) * 100

In [ ]:
# print the hyper curve fit

print(json.dumps([list(xvals),list(hyperline(xvals))]))
hCtT_X[i][-4:-1]

In [ ]:
hyperline

In [ ]:
plt.plot(xvals,smoothed_X[0],'.')

In [ ]:

plt.plot(xvals,deri,'--',alpha=0.8)


In [ ]:
# smooth 
smoothT = Pipeline([
    ('smooth', Smoother(stddev=2, windowlength=11, window='hanning')),
  
])
smoothed_X = smoothT.transform(X)
fix,ax =plt.subplots(1,1)
ax.plot(np.linspace(0,30,90),smoothed_X[0][1],'.')
print(json.dumps([list(np.linspace(0,30,90)),list(smoothed_X[0][1])]))

In [ ]:
# smooth 
smoothT = Pipeline([
    ('smooth', Smoother(stddev=2, windowlength=11, window='hanning')),
  ('normalize', Normalize(mode='mean', normalizeRange=(normStart, normEnd))),
])
smoothed_X = smoothT.transform(X)
fix,ax =plt.subplots(1,1)
ax.plot(np.linspace(0,30,90),smoothed_X[0][1],'.')
print(json.dumps([list(np.linspace(0,30,90)),list(smoothed_X[0][1])]))

In [ ]:
# smooth normalize and truncate
smoothT = Pipeline([
    ('smooth', Smoother(stddev=2, windowlength=11, window='hanning')),
  ('normalize', Normalize(mode='mean', normalizeRange=(normStart, normEnd))),
  ('truncate', Truncate(cutoffStart=cutoffStart, cutoffEnd=cutoffEnd, n=90)),
])
smoothed_X = smoothT.transform(X)
fix,ax =plt.subplots(1,1)
ax.plot(xvals,smoothed_X[0][1],'.')
print(json.dumps([list(xvals),list(smoothed_X[0][1])]))

In [ ]:
# the curve after S-G filter
smoothT = Pipeline([
    ('smooth', Smoother(stddev=2, windowlength=11, window='hanning')),
   ('normalize', Normalize(mode='mean', normalizeRange=(normStart, normEnd))),
    ('truncate', Truncate(cutoffStart=cutoffStart, cutoffEnd=cutoffEnd, n=90)),
    ('Derivitive', Derivitive(window=31, deg=3,deriv=0)),
])
smoothed_X = smoothT.transform(X)
fix,ax =plt.subplots(1,1)
ax.plot(xvals,-smoothed_X[0][1],'.')
print(json.dumps([list(xvals),list(-smoothed_X[0][1])]))

In [ ]:
len(smoothed_X[0][1])

In [ ]:
# the curve after S-G filter and get derivative
smoothT = Pipeline([
    ('smooth', Smoother(stddev=2, windowlength=11, window='hanning')),
   ('normalize', Normalize(mode='mean', normalizeRange=(normStart, normEnd))),
    ('truncate', Truncate(cutoffStart=cutoffStart, cutoffEnd=cutoffEnd, n=90)),
    ('Derivitive', Derivitive(window=31, deg=3,deriv=1)),
])
smoothed_X = smoothT.transform(X)
fix,ax =plt.subplots(1,1)
ax.plot(xvals,-smoothed_X[0][1],'.')
print(json.dumps([list(xvals),list(-smoothed_X[0][1])]))

In [ ]:
# the curve after S-G filter and get derivative negate
smoothT = Pipeline([
    ('smooth', Smoother(stddev=2, windowlength=11, window='hanning')),
   ('normalize', Normalize(mode='mean', normalizeRange=(normStart, normEnd))),
    ('truncate', Truncate(cutoffStart=cutoffStart, cutoffEnd=cutoffEnd, n=90)),
    ('Derivitive', Derivitive(window=31, deg=3,deriv=1)),
])
smoothed_X = smoothT.transform(X)
fix,ax =plt.subplots(1,1)
ax.plot(xvals,smoothed_X[0][1],'.')
print(json.dumps([list(xvals),list(smoothed_X[0][1])]))

In [ ]:
#different window 
window=25
# the curve after S-G filter and get derivative negate
smoothT = Pipeline([
    ('smooth', Smoother(stddev=2, windowlength=11, window='hanning')),
   ('normalize', Normalize(mode='mean', normalizeRange=(normStart, normEnd))),
    ('truncate', Truncate(cutoffStart=cutoffStart, cutoffEnd=cutoffEnd, n=90)),
    ('Derivitive', Derivitive(window=window, deg=3,deriv=0)),
])
smoothed_X = smoothT.transform(X)
fix,ax =plt.subplots(1,1)
ax.plot(xvals,smoothed_X[0][1],'.')
print('左图:', json.dumps([list(xvals),list(smoothed_X[0][1])]))

# the curve after S-G filter and get derivative negate
smoothT = Pipeline([
    ('smooth', Smoother(stddev=2, windowlength=11, window='hanning')),
   ('normalize', Normalize(mode='mean', normalizeRange=(normStart, normEnd))),
    ('truncate', Truncate(cutoffStart=cutoffStart, cutoffEnd=cutoffEnd, n=90)),
    ('Derivitive', Derivitive(window=window, deg=3,deriv=1)),
])
smoothed_X = smoothT.transform(X)
fix,ax =plt.subplots(1,1)
ax.plot(xvals,smoothed_X[0][1],'.')
print('右图:',json.dumps([list(xvals),list(smoothed_X[0][1])]))

In [ ]:
v = np.linspace(-600,0,2)
f = - v/10000000 * 1e6
r = - (v + 100)/10000000 * 1e6
c = f-r
plt.plot(v,f,v,r,v,c)

In [ ]:
# the curve after S-G filter and get derivative
smoothT = Pipeline([
    ('smooth', Smoother(stddev=2, windowlength=11, window='hanning')),
   ('normalize', Normalize(mode='mean', normalizeRange=(normStart, normEnd))),
    ('truncate', Truncate(cutoffStart=cutoffStart, cutoffEnd=cutoffEnd, n=90)),
    ('Derivitive', Derivitive(window=31, deg=3,deriv=1)),
])
smoothed_X = smoothT.transform(X)
fix,ax =plt.subplots(1,1)
ax.plot(xvals,smoothed_X[0][1],'.')
print(json.dumps([list(xvals),list(-smoothed_X[0][1])]))

In [ ]:
len(smoothed_X[0])

In [ ]:

X = smoothed_X[0]
t,gradient,pc = X
gradient = gradient 
heightlimit = np.quantile(np.absolute(gradient[0:-1] - gradient[1:]), 0.9)
peaks,props = signal.find_peaks(gradient,prominence=heightlimit,width= len(gradient) * 0.05, rel_height=0.5,height=heightlimit)

t=[5,30]

peak_pos,left_ips,peak_prominence,peak_width = (t[-1],t[-1],0,0)
sdAtRightIps,sdAt3min,sdAt5min,sdAt10min,sdAt15min,sdAtEnd = (0,0,0,0,0,0)

In [ ]:
if len(peaks) != 0:                
    tspan = t[-1]-t[0] # 25 分钟
    
    # 每个点的时间间隔， 25/90
    normalizer =  tspan / len(gradient) 
    
    # 找到最大prominence的峰的在signal.finde_peaks的结果里的index，这里是0    
    maxpeak_index = props['prominences'].argmax() 
    
    # 最大prominence的峰的prominence，0.013573826379022486
    peak_prominence = props['prominences'][maxpeak_index]  
    
    # 左半峰宽交叉点的时间： 10.5691635309分钟
    left_ips = props['left_ips'][maxpeak_index] * normalizer  + t[0] 

    pcMaxIdx = len(pc) - 1 # 最后一个点的index， 89
    
    startPosition = int(props['left_ips'][maxpeak_index]) # 左半分峰宽交叉点的index，20
    
    sStart = pc[startPosition] # 左半峰宽交叉点的纵坐标： 0.92553695440 

    # 左半峰宽交叉点往后5分钟的sd，0.224235150457， 用min函数防止越界
    sdAt5min = sStart - pc[min(startPosition + int(5 / normalizer), pcMaxIdx)] 
    
    

In [ ]:
#手动计算left_ips

# 找到最大prominence的峰的位置， 这里是35
peak_position = peaks[maxpeak_index]

# 峰的高度， 0.01528766747
peak_height = gradient[peak_position] 

# 用来计算半峰宽的高度： 0.00850075428579954 （峰高度 -  prominence * 0.5）
evaluation_point = peak_height - peak_prominence * 0.5

#初始化
manual_left_ips_index = peak_position

#从峰的位置，向左一个一个的找
for i in gradient[0:peak_position:][::-1]:
    # 如果找到了一个比峰高度小的点，停止循环
    if i < evaluation_point:        
        break
    # 否则，继续向左找，更新左半峰宽交叉点的index
    manual_left_ips_index = manual_left_ips_index - 1

#计算左半峰宽交叉点的时间： 10.83333333 分钟
manual_left_ips = manual_left_ips_index* normalizer  + t[0] 

In [ ]:
from scipy.optimize import least_squares

def findTimeVal(t,val,t0,dt):
    """
    t:   [.............]
    val: [.............]
    t0:       |      ; if t0 is less than 0, then start from 0
    dt:       |---|  ; must > 0
    return:  [.....]
    find the fragment of time series data,
    based on starting time t0 and time length to extract
    assuming t is an evenly spaced time series
    """
    t0idx = int((t0 - t[0]) / (t[-1]-t[0]) * len(val))
    t1idx = int((t0 +dt - t[0]) / (t[-1]-t[0]) * len(val))
    return val[max(0,t0idx):t1idx]

def hyper(p,x,y):
    "用来拟合的双曲线函数"
    return p[0]/(x+p[1]) +p[2] -y

t=[5,30]
# 左半峰宽交叉点的时间： 10.5691635309分钟 
left_ips = 10.5691635309
# 找到5-10.569分钟之间的pc的值来进行双曲线拟合
tofit = findTimeVal(t,pc,t[0],left_ips - t[0])
#这些数据对应的时间：
times = np.linspace(t[0],left_ips,len(tofit))

# 双曲线拟合
fitres = least_squares(hyper,x0=[5,5,0.5],
                args=(times,tofit))
#拟合的结果: [495.39542968, 166.48141007,  -1.84842431]; 对应了p[0],p[1],p[2]
fitpara = fitres.x

# 拟合的双曲线函数下移的%
offset = 0.05
# 下移的纵坐标值是拟合的区间最后一个点的纵坐标 * offset
# 相当于将p[2]下移了这个值
thresholdpara = fitpara - np.array([0,0,(tofit[-1]) * offset])


# 从左半峰宽交叉点的时间，向右找出了所有要找的点的纵坐标
# [0.92553695, 0.91645038, 0.90831233,，。。。 0.56696346, 0.56785669】
# 共70个点
tosearch = findTimeVal(t,pc,left_ips,t[-1])
# 要找的点的对应的时间
tosearchT = np.linspace(left_ips,t[-1],len(tosearch))

# 初始化thresholdCt为左半峰宽交叉点的时间
thresholdCt = left_ips
# 向右一个一个的找。在开始的时候，双曲线函数的纵坐标小于数据点的纵坐标，
for t,y in zip(tosearchT,tosearch):
    #如果delta大于0了，下移的后的双曲线函数的纵坐标大于数据点的纵坐标，停止循环
    delta = hyper(thresholdpara,t,y)
    if delta >0:
        break
    thresholdCt = t
# 最终的结果：thresholdCt = 11.97719515909565



In [ ]:
t

In [ ]:
times = np.arange(t[0],left_ips, (t[-1]-t[0]) / len(pc) )

In [ ]:
len(tofit)

In [ ]:
10.83333333+0.277

In [ ]:
min(gradient)

In [ ]:
len(gradient)

In [ ]:
pc

In [ ]:
peaks

In [ ]:
gradient[35]